Florida Panthers Hockey Club Historical Review (2017/18-2023/24)

YoY Metrics to Track:

~ Tickets
    ~ How have our biggest games improved? Smallest Games? Season Averages? Post-Season?
    ~ KPIs : Gross Revenue, Paid Tickets, ATP, Comp Ticket Conversion Rate
~ Attendance 
    ~ Once people buy do they show up? Single game show-rate? Plan show-rate? 
    ~ KPIs : Show Rate (by game/by tier/by ticket type)
~ Parking 
    ~ How do game factors influence parking numbers?
    ~ KPIs: Prepaid (difference with Seatgeek) v On-Site, People per Car, Per Cap, # of Cars
~ F&B 
    ~ How do game factors influence F&B numbers? Change by location? Parnerships?
    ~ KPIs: Gross Revenue, Per Cap, Avg Item Price
~ Merch
    ~ How do game factors influence merch numbers? Change by location?
    ~ KPIs: Gross Revenue, Per Cap, Avg Item Price
~ All-In Total
    ~ How much does it cost for someone to attend a Panthers game?

Other Factors to Consider:

~ COVID-19
~ On-ice performance
~ Growth of STM base

Data Sources Used:

~ cth_historical_all_1718_2223
~ cth_v_ticket_2324

Models Created: 

~ 

Future Considerations:

~ 

In [4]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [5]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

Part I ~ Tickets

In [ ]:
# all regular season games since 2017-18 (excluding covid season) paid seats, comp seats, gross revenue, ATP
    # quick pivot by tier, opponent, ticket type
    # growth of STMs
    # bowl analysis
    # num of each tier of games by season, points % of team each season (first half? if they made the playoffs?)
    # YoY Growth by each metric - model for equation that shows growth to predict into future on current trajectory?

In [33]:
# import data

q = """
WITH all_games AS
    (SELECT season, date(event_date) as event_date, ticket_type,
        CASE
            when pc_one in ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8') then 'Lowers'
            when pc_one in ('K', 'L', 'M') then 'Clubs'
            when pc_one in ('N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y') then 'Uppers'
            when pc_one in ('U', 'V', 'W') then 'Suites'
            when pc_one in ('X', 'Z') then 'Premium'
        end as arena_level_internal,
        sum(gross_revenue) as gross_revenue,
        sum(paid_seats) as paid_seats, sum(comp_seats) as comp_seats
    FROM custom.cth_historical_all_1718_2223
    WHERE season != '2020-21'
    GROUP BY season, event_date, ticket_type, arena_level_internal
    UNION ALL
    SELECT '2023-24' as season, date(event_datetime) as event_date, ticket_type, arena_level_internal, sum(gross_revenue) as gross_revenue,
       sum(paid_seats) as paid_seats, sum(comp_seats) as comp_seats
    FROM custom.cth_v_ticket_2324
    GROUP BY season, event_date, ticket_type, arena_level_internal)
SELECT all_games.*, tier, abbreviation
FROM all_games
LEFT JOIN custom.cth_game_descriptions
    on all_games.event_date = cth_game_descriptions.event_date
"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [35]:
df_pivot = df.groupby(by = ['season', 'event_date'], axis = 0)[['gross_revenue','paid_seats','comp_seats']].sum()
pd.concat()

pandas.core.frame.DataFrame

In [32]:
df_tier = df.groupby(by = ['tier', 'season'], axis = 0)[['gross_revenue','paid_seats','comp_seats']].mean()
df_abbr = df.groupby(by = ['abbreviation', 'season'], axis = 0)[['gross_revenue','paid_seats','comp_seats']].mean()
df_abbr

gross_revenue  paid_seats  comp_seats
abbreviation season                                        
ANA          2017-18     485819.030      9759.0      1099.0
             2018-19     500259.030      8703.0       733.0
             2019-20     473683.400      9433.0      2703.0
             2021-22     679660.730     13880.0      2324.0
             2022-23     736829.120     14543.0      1952.0
...                             ...         ...         ...
WSH          2018-19     723644.220     12978.0      1907.0
             2019-20     707647.500     12290.0      2266.0
             2021-22    1166180.092     15854.4       704.8
             2022-23     693485.390     12851.0       965.0
             2023-24    1327419.655     17817.0       954.5

[180 rows x 3 columns]

Part II ~ Attendance

In [ ]:
# all regular season games since 2017-18 (excluding covid season?) paid attendance, comp attendance, paid show rate, comp show rate
    # quick pivot by tier, ticket type
    # num of plan tickets sold/scanned by another account
    # bowl analysis
    # num of each tier of games by season, points % of team each season (first half? if they made the playoffs?)
    # YoY Growth by each metric - model for equation that shows growth to predict into future on current trajectory?

Part III ~ Parking

In [ ]:
# all regular season games since 2017-18 (excluding covid season?) # cars, prepaid v onsite breakdown, gross revenue
    # quick pivot by tier, day of week/start time
    # prepaid v onsite analysis/growth by season
    # num of each tier of games by season, points % of team each season (first half? if they made the playoffs?)
    # YoY Growth by each metric - model for equation that shows growth to predict into future on current trajectory?

Part IV ~ F&B 

In [ ]:
# all regular season games since 2017-18 (excluding covid season?) gross revenue, # items, location in arena
    # quick pivot by tier, day of week/start time
    # by location/bowl level growth
    # parnership growth
    # num of each tier of games by season, points % of team each season (first half? if they made the playoffs?)
    # YoY Growth by each metric - model for equation that shows growth to predict into future on current trajectory?

Part V ~ Merch

In [ ]:
# all regular season games since 2017-18 (excluding covid season?) gross revenue, # items, location in arena
    # quick pivot by tier, trimester?
    # num of each tier of games by season, points % of team each season (first half? if they made the playoffs?)
    # YoY Growth by each metric - model for equation that shows growth to predict into future on current trajectory?

Part VI ~ All-In Total

In [ ]:
# aggregate all previous sections
    # how much additional revenue have we generated in total?